# Analyse failed trades

Analyse failed trades over multiple trade executors.

 - Show failed and repaired and fix trades
  - Analyse slippage parameters and other causes of failure



In [1]:
monitored_executors = [
    "https://enzyme-polygon-eth-usdc.tradingstrategy.ai",
    "https://enzyme-polygon-matic-usdc.tradingstrategy.ai",
    "https://enzyme-polygon-multipair.tradingstrategy.ai",
]



In [2]:
import requests
from tradeexecutor.state.state import State

total_downloaded = 0

def download_state(url) -> State:
    global total_downloaded
    resp = requests.get(f"{url}/state")
    state = State.read_json_blob(resp.text)
    total_downloaded += len(resp.content)
    return state

states = [download_state(url)for url in monitored_executors]


print(f"Downloaded {total_downloaded:,} bytes")

/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:184: RuntimeWarning: `NoneType` object value of non-optional type initial_cash detected when decoding TradeSummary.
  warnings.warn(


Downloaded 1,509,517 bytes


In [3]:
from tradeexecutor.state.trade import TradeExecution
from itertools import chain

# Get all trades across different trade executors
all_trades = chain(*[state.portfolio.get_all_trades() for state in states])

t: TradeExecution
failed_trades = [t for t in all_trades if t.is_failed() or t.is_repaired() or t.is_repair_trade()]

print(f"Total {len(failed_trades)} failed and repaired trades")

Total 62 failed and repaired trades


In [4]:
import pandas as pd
from tradeexecutor.analysis.slippage import display_slippage

display_slippage(failed_trades)

,Flags,Position,Ticker,Started,Executed,Lag,Slippage tolerance,Notes,Failure reason
1,REP,#1,WETH-USDC,2023-07-10 19:00,2023-07-11 07:39,7.0,0.01,"Repaired at 2023-07-11 07:39, by #4",Too little received                           ...
4,FIX,#1,WETH-USDC,,2023-07-11 07:39,,,Repairing trade #1,
2,REP,#2,WETH-USDC,2023-07-10 20:00,2023-07-11 07:39,6.0,0.01,"Repaired at 2023-07-11 07:39, by #5",Too little received                           ...
5,FIX,#2,WETH-USDC,,2023-07-11 07:39,,,Repairing trade #2,
5,REP,#3,WMATIC-USDC,2023-06-30 04:00,2023-07-03 07:45,11.0,0.01,"Repaired at 2023-07-03 07:45, by #8",Too little received                           ...
...,...,...,...,...,...,...,...,...,...
58,FIX,#27,XSGD-USDC,,2023-07-11 07:41,,,Repairing trade #42,
43,REP,#28,XSGD-USDC,2023-07-08 01:01,2023-07-11 07:41,69.0,0.01,"Repaired at 2023-07-11 07:41, by #59",execution reverted: __preProcessCoI: Non-recei...
59,FIX,#28,XSGD-USDC,,2023-07-11 07:41,,,Repairing trade #43,
46,REP,#30,XSGD-USDC,2023-07-11 07:00,2023-07-11 07:41,7.0,0.01,"Repaired at 2023-07-11 07:41, by #60",execution reverted: __preProcessCoI: Non-recei...
